In [58]:
import os
import imageio
import rasterio
from skimage.transform import resize
from PIL import Image, ImageDraw, ImageFont
import numpy as np

# Specify the main directory containing the subfolders with "Color" folders
main_directory = '/Users/Feifei/Box/PlanetClassification/Tuotuo'

# Specify the path to the output GIF file
output_gif = '/Users/Feifei/Box/PlanetClassification/Tuotuo/gifs/output.gif'


# Function to recursively search for GeoTIFF files in the "Color" folders
def find_geotiff_files(directory):
    geotiff_files = []
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            if dir_name == 'Color':
                color_folder_path = os.path.join(root, dir_name)
                geotiff_files.extend(find_geotiff_files_in_folder(color_folder_path))
    return geotiff_files


# Function to find GeoTIFF files within a specific "Color" folder
def find_geotiff_files_in_folder(folder_path):
    geotiff_files = []
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.tif') or file_name.endswith('.tiff'):
            geotiff_files.append(os.path.join(folder_path, file_name))
    return geotiff_files



# Function to convert GeoTIFF file to RGB array and add filename as text overlay
def convert_geotiff_to_rgb_array(geotiff_file):
    with rasterio.open(geotiff_file) as dataset:
        file_name = os.path.splitext(os.path.basename(geotiff_file))[0]

        bands = dataset.read()

        if "PS" in file_name:
            band1 = bands[3, :, :]
            band2 = bands[2, :, :]
            band3 = bands[1, :, :]
        elif "RE" in file_name:
            band1 = bands[4, :, :]
            band2 = bands[2, :, :]
            band3 = bands[1, :, :]
        else:
            return None

        bands = [band1, band2, band3]

        bands = [band.astype(np.float32) for band in bands]
        bands = [(band - np.min(band)) / (np.max(band) - np.min(band)) for band in bands]

        rgb_array = np.transpose(bands, (1, 2, 0))

        image = Image.fromarray((rgb_array * 255).astype(np.uint8))
        draw = ImageDraw.Draw(image)
        font = ImageFont.truetype("arial.ttf", 100)  # Adjust the font, size, and bold as desired
        text_bbox = draw.textbbox((10, 10), file_name, font=font)
        draw.text((10, 10), file_name, (255, 255, 255), font=font)

        return np.array(image)


# Create a list to store the frames of the GIF
gif_frames = []

# Find GeoTIFF files within the "Color" folders
geotiff_files = find_geotiff_files(main_directory)

# Loop through the GeoTIFF files and convert to RGB array
for geotiff_file in geotiff_files:
    rgb_array = convert_geotiff_to_rgb_array(geotiff_file)
    if rgb_array is not None:
        resized_rgb_array = resize(rgb_array, (512, 512))
        gif_frames.append(resized_rgb_array)

# Save the frames as a GIF file
imageio.mimsave(output_gif, gif_frames, format='GIF', duration=2)  # Adjust duration as desired


Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from float64 to uint8. Range [0

In [11]:
import os
import imageio
import numpy as np
from skimage.transform import resize

river = "Mahajamba_3"

# Using an f-string:
main_directory = f"C:\\Users\\Feifei\\Box\\BR_remote_sensing\\Madagascar\\{river}\\output_annual\\{river}"
output_images_folder = f"C:\\Users\\Feifei\\Box\\BR_remote_sensing\\Madagascar\\{river}\\output_annual\\{river}\\images_png"



# Desired size for the resized images
image_size = (512, 412)

# Make sure the folder exists
if not os.path.exists(output_images_folder):
    os.makedirs(output_images_folder)

def find_tiff_files(directory):
    """Recursively find TIFF files in 'mask' folders."""
    tiff_files = []
    for root, dirs, files in os.walk(directory):
        for dir_name in dirs:
            if dir_name.lower() == 'mask':
                mask_folder_path = os.path.join(root, dir_name)
                tiff_files.extend(find_tiff_files_in_folder(mask_folder_path))
    return tiff_files

def find_tiff_files_in_folder(folder_path):
    """Return all TIFF files from a given folder."""
    tiff_files = []
    for file_name in os.listdir(folder_path):
        if file_name.lower().endswith('.tif') or file_name.lower().endswith('.tiff'):
            tiff_files.append(os.path.join(folder_path, file_name))
    return tiff_files

# Collect all TIFF files
tiff_files = find_tiff_files(main_directory)

# Counter for how many we skip
skipped_images = 0

# Loop over each TIFF, process it, and save as PNG
for tiff_file in tiff_files:
    # Read the TIFF image
    image = imageio.imread(tiff_file)
    
    # Resize the image
    resized_image = resize(image, image_size, anti_aliasing=True, preserve_range=True)
    
    # Convert to binary mask (0/1) based on > 0
    binary_image = (resized_image > 0).astype(np.uint8)  # 0 or 1
    
    # Scale to 0 or 255
    scaled_image = binary_image * 255
    
    # Check if the entire image is blank (all zeros)
    if np.all(scaled_image == 0):
        # Skip saving this image and increment the counter
        skipped_images += 1
        continue
    
    # Keep the same base filename
    base_name = os.path.splitext(os.path.basename(tiff_file))[0]
    
    # Save as PNG
    output_image_path = os.path.join(output_images_folder, f"{base_name}.png")
    imageio.imwrite(output_image_path, scaled_image)

print(f"Done! PNG images are saved in: {output_images_folder}")
print(f"Skipped {skipped_images} blank images.")




C:\Users\Feifei\AppData\Local\Temp\ipykernel_4904\1364768478.py:48: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(tiff_file)


Done! PNG images are saved in: C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\Mahajamba_3\output_annual\Mahajamba_3\images_png
Skipped 4 blank images.


In [5]:
import os
import imageio

# Folder containing your PNG images
png_folder = r"C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\20903040201\output_annual\20903040201\images_png"

# Desired output GIF path
output_gif = r"C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\20903040201_gif.gif"

# Collect all PNG files (sort them alphabetically or by modified time if needed)
png_files = [f for f in os.listdir(png_folder) if f.lower().endswith('.png')]
png_files.sort()  # Optional: ensures a consistent order

# Create a list of frames (numpy arrays) for the GIF
gif_frames = []
for png_file in png_files:
    png_path = os.path.join(png_folder, png_file)
    frame = imageio.imread(png_path)
    gif_frames.append(frame)

# Create a GIF with each image shown for 2 seconds
durations = [1000] * len(gif_frames)  # 10 seconds per frame
imageio.mimsave(output_gif, gif_frames, format='GIF', duration=durations)


print(f"GIF created:\n{output_gif}")


C:\Users\Feifei\AppData\Local\Temp\ipykernel_4904\3034184123.py:18: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frame = imageio.imread(png_path)


GIF created:
C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\20903040201_gif.gif


In [12]:
import os
import imageio
import numpy as np

png_folder = r"C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\Mahajamba_3\output_annual\Mahajamba_3\images_png"


# Output path for the overlapped image
overlap_output = r"C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\Mahajamba_3\Mahajamba_3_overlapped.png"

# Collect all PNG files
png_files = [f for f in os.listdir(png_folder) if f.lower().endswith('.png')]
png_files.sort()  # ensure a consistent order (alphabetic)

# Check that there are at least 2 images
if len(png_files) < 2:
    raise ValueError("Not enough PNG files in the folder to overlap (need at least two).")

# Identify the first and last file
first_file = os.path.join(png_folder, png_files[0])   # earliest
last_file  = os.path.join(png_folder, png_files[-1])  # latest

# Read them as arrays
first_img = imageio.imread(first_file)   # shape: (H, W)
last_img  = imageio.imread(last_file)    # shape: (H, W)

# Ensure both images have the same shape
if first_img.shape != last_img.shape:
    raise ValueError("First and last images have different shapes; can't overlap them directly.")

# Convert 2D -> 3D color arrays: [H, W, 3]
#   first_rgb will be green where first_img == 255, black where == 0
#   last_rgb  will be red   where last_img  == 255, black where == 0

H, W = first_img.shape
first_rgb = np.zeros((H, W, 3), dtype=np.uint8) 
last_rgb  = np.zeros((H, W, 3), dtype=np.uint8)

# Wherever the first image is 'on' (255), make it green = [0, 255, 0]
first_rgb[first_img == 255] = [0, 255, 0]

# Wherever the last image is 'on' (255), make it red = [255, 0, 0]
last_rgb[last_img == 255] = [255, 0, 0]

# Now overlap them with alpha blending
# overlap = alpha * last_rgb + (1 - alpha) * first_rgb
alpha = 0.5

# Convert to float for blending
first_float = first_rgb.astype(np.float32)
last_float  = last_rgb.astype(np.float32)

overlapped_float = alpha * last_float + (1 - alpha) * first_float
overlapped_img   = overlapped_float.astype(np.uint8)

# Save the overlapped image
imageio.imwrite(overlap_output, overlapped_img)

print("Overlapped image saved to:", overlap_output)



Overlapped image saved to: C:\Users\Feifei\Box\BR_remote_sensing\Madagascar\Mahajamba_3\Mahajamba_3_overlapped.png


C:\Users\Feifei\AppData\Local\Temp\ipykernel_4904\1688995756.py:24: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  first_img = imageio.imread(first_file)   # shape: (H, W)
C:\Users\Feifei\AppData\Local\Temp\ipykernel_4904\1688995756.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  last_img  = imageio.imread(last_file)    # shape: (H, W)
